# Backward Propagation with Mini-Batch Stochastic Gradiant Descent

## Spesifikasi
- Jumlah hidden layer maksimal 10
- Jumlah node dalam setiap hidden layer dapat bervariasi
- Fully-connected layer
- Fungsi aktivasi berupa sigmoid untuk semua hidden layer maupun output layer
- Node output berjumlah 1
- Program memberikan pilihan untuk menggunakan momentum atau tidak
- Program mengimplementasikan mini-batch stochastic gradient descent
- Implementasi incremental dengan setting batch-size=1 dan implementasi batch dengan setting batchsize=jumlah
data.

Algoritma ini akan diuji dengan data weather (tennis) yang diambil dari Weka

In [2]:
from scipy.io import arff
import numpy as np
import pandas as pd
import math
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras import optimizers
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [3]:
class NeuralNetwork():
    def __init__(self, 
                 input_nodes,
                 hidden_nodes=[],
                 output_nodes=1,
                 batch_size=4,
                 learning_rate=1e-4,
                 momentum=0,
                 threshold=0.5):
        assert(input_nodes >= 1)
        assert(0 <= len(hidden_nodes) <= 10)
        assert(batch_size >= 1)
        
        self.layers = self._init_layers(input_nodes, hidden_nodes, output_nodes)
        self.batch_size = batch_size
        self.learning_rate = learning_rate
        self.momentum = momentum
        self.threshold = threshold
        
        self.params_values = self._init_weights()

        
    def _init_layers(self, input_nodes, hidden_nodes, output_nodes):
        layers = []
        layers.append(input_nodes)
        for hidden_layer in hidden_nodes:
            layers.append(hidden_layer)
        layers.append(output_nodes)
        
        return layers
    
    def _init_grads_values(self):
        grads_values = {}
        for idx in range(1, len(self.layers)):
            layer_input_size = self.layers[idx-1]
            layer_output_size = self.layers[idx]
            grads_values['W' + str(idx)] = np.zeros([layer_output_size, layer_input_size])
            grads_values['b' + str(idx)] = np.zeros([layer_output_size])
        return grads_values
        
    def _init_weights(self):
        """
        Initiate weights and bias weights for the neural network
        """
        params_values = {}
        for idx in range(len(self.layers)-1):
            layer_input_size = self.layers[idx]
            layer_output_size = self.layers[idx+1]
            
            # Weight
            params_values['W' + str(idx+1)] = np.random.randn(layer_output_size, layer_input_size) * 0.1
            
            # Bias Weight
            params_values['b' + str(idx+1)] = np.random.randn(layer_output_size) * 0.1
            
        return params_values
    
    
    def _single_layer_feed_forward(self, A_prev, W_curr, b_curr):
        """
        Feed forward for single layer in neural network
        """
        Z_curr = np.dot(W_curr, A_prev) + b_curr
        return self._sigmoid(Z_curr), Z_curr
        
        
    def _full_feed_forward(self, X):
        memory = {}
        A_curr = X
        
        for idx in range(len(self.layers)-1):
            A_prev = A_curr
            
            W_curr = self.params_values['W' + str(idx+1)]
            b_curr = self.params_values['b' + str(idx+1)]
            
            A_curr, Z_curr = self._single_layer_feed_forward(A_prev, W_curr, b_curr)
            
            memory['A' + str(idx)] = A_prev
            memory['Z' + str(idx+1)] = Z_curr
            
        memory['A' + str(len(self.layers)-1)] = A_curr
        return A_curr, memory
        
    
    def _single_layer_backward_prop(self, dA_curr, W_curr, b_curr, Z_curr, A_prev):
        
        dZ_curr = dA_curr * self._sigmoid_backward(Z_curr)
        dW_curr = np.outer(dZ_curr, A_prev)
        db_curr = dZ_curr
        dA_prev = np.dot(W_curr.T, dZ_curr)
        
        return dA_prev, dW_curr, db_curr
    
    
    def _full_backward_prop(self, y_hat, y, memory):
        grads_values = {}
        
        dA_curr = y - y_hat
        
        for layer_idx_prev in range(len(self.layers)-2, -1, -1):
            layer_idx_curr = layer_idx_prev + 1
            
            A_curr = memory['A' + str(layer_idx_curr)]
            A_prev = memory['A' + str(layer_idx_prev)]
            Z_curr = memory['Z' + str(layer_idx_curr)]
            W_curr = self.params_values['W' + str(layer_idx_curr)]
            b_curr = self.params_values['b' + str(layer_idx_curr)]
            
            dA_prev, dW_curr, db_curr = self._single_layer_backward_prop(dA_curr, W_curr, b_curr, Z_curr, A_prev)
            
            grads_values['dW' + str(layer_idx_curr)] = dW_curr
            grads_values['db' + str(layer_idx_curr)] = db_curr
            
        return grads_values
    
    
    def _update(self, grads_values):
        for layer_idx in range(1, len(self.layers)):
            self.params_values['W' + str(layer_idx)] += self.learning_rate * grads_values['dW' + str(layer_idx)] + self.momentum * self.params_values['W' + str(layer_idx)]
            self.params_values['b' + str(layer_idx)] += self.learning_rate * grads_values['db' + str(layer_idx)] + self.momentum * self.params_values['b' + str(layer_idx)]

            
    def _sigmoid(self, weighted_sum):
        return 1/(1+np.exp(-weighted_sum))    
    
    
    def _sigmoid_backward(self, y):
        sigmoid = self._sigmoid(y)
        return sigmoid * (1 - sigmoid)
    
        
    def _calc_error(self, output, target):
        return 0.5 * ((output - target) ** 2)
    
    
    def _threshold(self, x):
        return 1 if x > self.threshold else 0
    
    
    def _calc_accuracy(self, output, target):
        count_correct = 0
        for i in range(len(output)):
            if self._threshold(output[i]) == target:
                count_correct += 1
        return count_correct / len(output)
    
    
    def train(self, X, y, epochs):
        assert(len(X) == len(y))
        cost_history = []
        accuracy_history = []
        
        for epoch_idx in range(epochs):
            epoch_accuracy = 0
            epoch_loss = 0
            n_batch = 0
            for batch_start_idx in range(0, len(X), self.batch_size):
                n_batch += 1
                batch_accuracy = 0
                batch_loss = 0
                n_data_batch = 0
                y_hat, cache = None, None
                batch_grads_values = self._init_grads_values()
                for j in range(batch_start_idx, batch_start_idx + self.batch_size):
                    if j >= len(X):
                        break
                    n_data_batch += 1
                    y_hat, cache = self._full_feed_forward(X[j])
                    batch_grads_values = self._full_backward_prop(y_hat, y[j], cache)               
                    batch_accuracy += self._calc_accuracy(y_hat, y[j])
                    batch_loss += self._calc_error(y_hat, y[j])
                    self._update(batch_grads_values)
                epoch_accuracy += float(batch_accuracy) / float(n_data_batch)
                epoch_loss += float(batch_loss) / float(n_data_batch)
                
            acc = float(epoch_accuracy) / float(n_batch)
            accuracy_history.append(float(epoch_accuracy) / float(n_batch))
            cost = float(epoch_loss) / float(n_batch)
            cost_history.append(float(epoch_loss / n_batch))
            
            print("Epoch {}/{}, Loss={}, Accuracy={}".format(epoch_idx, epochs, cost, acc))
            
        return self.params_values, cost_history, accuracy_history
    
    def predict(self, X):
        y_preds = []
        for x in X:
            y_hat, cache = self._full_feed_forward(x)
            y_preds.append(y_hat)
            
        return y_preds

In [5]:
dataset = arff.loadarff('../data/weather.arff')
df = pd.DataFrame(dataset[0])

STR_COLUMNS = ['outlook', 'windy', 'play']

for column in STR_COLUMNS:
    df[column] = df[column].str.decode('utf-8')
    
df_encoded = pd.get_dummies(df, columns=['outlook', 'windy'])
df_encoded = pd.get_dummies(df_encoded, columns=['play'], drop_first=True)
df_encoded.temperature = df_encoded.temperature * 0.01
df_encoded.humidity = df_encoded.humidity * 0.01

X = df_encoded.iloc[:, :7].values
y = df_encoded.iloc[:, 7:].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=17779)

model = NeuralNetwork(input_nodes=7, hidden_nodes=[16, 32, 16, 8], output_nodes=1, batch_size=1, learning_rate=0.01, momentum=0.0)
param, costs, accs = model.train(X_train, y_train, epochs=1000)

print(param)
print()
print(costs)
print()
print(accs)

plt.plot(np.array(costs).mean(axis=1))
plt.ylabel('Cost')
plt.show()

plt.plot(accs)
plt.ylabel('Accuracy')
plt.show()

Epoch 0/1000, Loss=0.12908484401693596, Accuracy=0.3333333333333333
Epoch 1/1000, Loss=0.12831563856284572, Accuracy=0.3333333333333333
Epoch 2/1000, Loss=0.1275592393909948, Accuracy=0.3333333333333333
Epoch 3/1000, Loss=0.12681580490723893, Accuracy=0.3333333333333333
Epoch 4/1000, Loss=0.12608561529266318, Accuracy=0.3333333333333333
Epoch 5/1000, Loss=0.12536908113084175, Accuracy=0.4166666666666667
Epoch 6/1000, Loss=0.12466674594628507, Accuracy=0.6666666666666666
Epoch 7/1000, Loss=0.12397928172650781, Accuracy=0.6666666666666666
Epoch 8/1000, Loss=0.12330747694592004, Accuracy=0.6666666666666666
Epoch 9/1000, Loss=0.12265221716877078, Accuracy=0.6666666666666666
Epoch 10/1000, Loss=0.12201445892376382, Accuracy=0.6666666666666666
Epoch 11/1000, Loss=0.12139519814085416, Accuracy=0.6666666666666666
Epoch 12/1000, Loss=0.12079543494236389, Accuracy=0.6666666666666666
Epoch 13/1000, Loss=0.12021613691683064, Accuracy=0.6666666666666666
Epoch 14/1000, Loss=0.11965820312900423, Accu

Epoch 130/1000, Loss=0.11144404321671093, Accuracy=0.6666666666666666
Epoch 131/1000, Loss=0.11144327602467376, Accuracy=0.6666666666666666
Epoch 132/1000, Loss=0.11144250567010407, Accuracy=0.6666666666666666
Epoch 133/1000, Loss=0.11144173204599557, Accuracy=0.6666666666666666
Epoch 134/1000, Loss=0.11144095505835792, Accuracy=0.6666666666666666
Epoch 135/1000, Loss=0.11144017462493712, Accuracy=0.6666666666666666
Epoch 136/1000, Loss=0.11143939067405223, Accuracy=0.6666666666666666
Epoch 137/1000, Loss=0.11143860314353736, Accuracy=0.6666666666666666
Epoch 138/1000, Loss=0.11143781197978118, Accuracy=0.6666666666666666
Epoch 139/1000, Loss=0.11143701713685394, Accuracy=0.6666666666666666
Epoch 140/1000, Loss=0.11143621857571508, Accuracy=0.6666666666666666
Epoch 141/1000, Loss=0.11143541626349368, Accuracy=0.6666666666666666
Epoch 142/1000, Loss=0.11143461017283607, Accuracy=0.6666666666666666
Epoch 143/1000, Loss=0.11143380028131379, Accuracy=0.6666666666666666
Epoch 144/1000, Loss

Epoch 259/1000, Loss=0.11131484564158611, Accuracy=0.6666666666666666
Epoch 260/1000, Loss=0.11131360678494077, Accuracy=0.6666666666666666
Epoch 261/1000, Loss=0.11131236413176783, Accuracy=0.6666666666666666
Epoch 262/1000, Loss=0.11131111767496638, Accuracy=0.6666666666666666
Epoch 263/1000, Loss=0.1113098674073459, Accuracy=0.6666666666666666
Epoch 264/1000, Loss=0.11130861332162552, Accuracy=0.6666666666666666
Epoch 265/1000, Loss=0.11130735541043224, Accuracy=0.6666666666666666
Epoch 266/1000, Loss=0.11130609366630007, Accuracy=0.6666666666666666
Epoch 267/1000, Loss=0.11130482808166821, Accuracy=0.6666666666666666
Epoch 268/1000, Loss=0.11130355864887971, Accuracy=0.6666666666666666
Epoch 269/1000, Loss=0.11130228536017989, Accuracy=0.6666666666666666
Epoch 270/1000, Loss=0.11130100820771456, Accuracy=0.6666666666666666
Epoch 271/1000, Loss=0.11129972718352821, Accuracy=0.6666666666666666
Epoch 272/1000, Loss=0.11129844227956227, Accuracy=0.6666666666666666
Epoch 273/1000, Loss=

Epoch 378/1000, Loss=0.11113769802604336, Accuracy=0.6666666666666666
Epoch 379/1000, Loss=0.11113591441036329, Accuracy=0.6666666666666666
Epoch 380/1000, Loss=0.1111341248699937, Accuracy=0.6666666666666666
Epoch 381/1000, Loss=0.11113232937091716, Accuracy=0.6666666666666666
Epoch 382/1000, Loss=0.11113052787881188, Accuracy=0.6666666666666666
Epoch 383/1000, Loss=0.1111287203590526, Accuracy=0.6666666666666666
Epoch 384/1000, Loss=0.11112690677671151, Accuracy=0.6666666666666666
Epoch 385/1000, Loss=0.11112508709655881, Accuracy=0.6666666666666666
Epoch 386/1000, Loss=0.11112326128306367, Accuracy=0.6666666666666666
Epoch 387/1000, Loss=0.11112142930039513, Accuracy=0.6666666666666666
Epoch 388/1000, Loss=0.11111959111242291, Accuracy=0.6666666666666666
Epoch 389/1000, Loss=0.11111774668271819, Accuracy=0.6666666666666666
Epoch 390/1000, Loss=0.111115895974555, Accuracy=0.6666666666666666
Epoch 391/1000, Loss=0.1111140389509106, Accuracy=0.6666666666666666
Epoch 392/1000, Loss=0.11

Epoch 509/1000, Loss=0.11083783894921097, Accuracy=0.6666666666666666
Epoch 510/1000, Loss=0.11083488786114286, Accuracy=0.6666666666666666
Epoch 511/1000, Loss=0.11083192406499957, Accuracy=0.6666666666666666
Epoch 512/1000, Loss=0.11082894749109261, Accuracy=0.6666666666666666
Epoch 513/1000, Loss=0.11082595806942303, Accuracy=0.6666666666666666
Epoch 514/1000, Loss=0.1108229557296798, Accuracy=0.6666666666666666
Epoch 515/1000, Loss=0.11081994040123777, Accuracy=0.6666666666666666
Epoch 516/1000, Loss=0.11081691201315601, Accuracy=0.6666666666666666
Epoch 517/1000, Loss=0.11081387049417601, Accuracy=0.6666666666666666
Epoch 518/1000, Loss=0.11081081577271962, Accuracy=0.6666666666666666
Epoch 519/1000, Loss=0.11080774777688729, Accuracy=0.6666666666666666
Epoch 520/1000, Loss=0.11080466643445612, Accuracy=0.6666666666666666
Epoch 521/1000, Loss=0.11080157167287792, Accuracy=0.6666666666666666
Epoch 522/1000, Loss=0.11079846341927722, Accuracy=0.6666666666666666
Epoch 523/1000, Loss=

Epoch 674/1000, Loss=0.11011625631520472, Accuracy=0.6666666666666666
Epoch 675/1000, Loss=0.11010996441077824, Accuracy=0.6666666666666666
Epoch 676/1000, Loss=0.11010364217779212, Accuracy=0.6666666666666666
Epoch 677/1000, Loss=0.11009728945626752, Accuracy=0.6666666666666666
Epoch 678/1000, Loss=0.11009090608535092, Accuracy=0.6666666666666666
Epoch 679/1000, Loss=0.11008449190330888, Accuracy=0.6666666666666666
Epoch 680/1000, Loss=0.11007804674752336, Accuracy=0.6666666666666666
Epoch 681/1000, Loss=0.11007157045448678, Accuracy=0.6666666666666666
Epoch 682/1000, Loss=0.11006506285979696, Accuracy=0.6666666666666666
Epoch 683/1000, Loss=0.11005852379815234, Accuracy=0.6666666666666666
Epoch 684/1000, Loss=0.11005195310334685, Accuracy=0.6666666666666666
Epoch 685/1000, Loss=0.11004535060826502, Accuracy=0.6666666666666666
Epoch 686/1000, Loss=0.11003871614487698, Accuracy=0.6666666666666666
Epoch 687/1000, Loss=0.11003204954423357, Accuracy=0.6666666666666666
Epoch 688/1000, Loss

Epoch 809/1000, Loss=0.10891504585802553, Accuracy=0.6666666666666666
Epoch 810/1000, Loss=0.10890279477524585, Accuracy=0.6666666666666666
Epoch 811/1000, Loss=0.10889048182861229, Accuracy=0.6666666666666666
Epoch 812/1000, Loss=0.10887810669501218, Accuracy=0.6666666666666666
Epoch 813/1000, Loss=0.10886566904982847, Accuracy=0.6666666666666666
Epoch 814/1000, Loss=0.10885316856693711, Accuracy=0.6666666666666666
Epoch 815/1000, Loss=0.10884060491870412, Accuracy=0.6666666666666666
Epoch 816/1000, Loss=0.10882797777598362, Accuracy=0.6666666666666666
Epoch 817/1000, Loss=0.10881528680811463, Accuracy=0.6666666666666666
Epoch 818/1000, Loss=0.10880253168291922, Accuracy=0.6666666666666666
Epoch 819/1000, Loss=0.10878971206669986, Accuracy=0.6666666666666666
Epoch 820/1000, Loss=0.1087768276242373, Accuracy=0.6666666666666666
Epoch 821/1000, Loss=0.10876387801878819, Accuracy=0.6666666666666666
Epoch 822/1000, Loss=0.10875086291208323, Accuracy=0.6666666666666666
Epoch 823/1000, Loss=

Epoch 937/1000, Loss=0.10671751239196033, Accuracy=0.6666666666666666
Epoch 938/1000, Loss=0.10669419984241453, Accuracy=0.6666666666666666
Epoch 939/1000, Loss=0.10667077203617725, Accuracy=0.6666666666666666
Epoch 940/1000, Loss=0.1066472284752202, Accuracy=0.6666666666666666
Epoch 941/1000, Loss=0.10662356866077327, Accuracy=0.6666666666666666
Epoch 942/1000, Loss=0.10659979209334486, Accuracy=0.6666666666666666
Epoch 943/1000, Loss=0.10657589827274282, Accuracy=0.6666666666666666
Epoch 944/1000, Loss=0.10655188669809523, Accuracy=0.6666666666666666
Epoch 945/1000, Loss=0.10652775686787196, Accuracy=0.6666666666666666
Epoch 946/1000, Loss=0.10650350827990623, Accuracy=0.6666666666666666
Epoch 947/1000, Loss=0.10647914043141693, Accuracy=0.6666666666666666
Epoch 948/1000, Loss=0.10645465281903059, Accuracy=0.6666666666666666
Epoch 949/1000, Loss=0.10643004493880447, Accuracy=0.6666666666666666
Epoch 950/1000, Loss=0.10640531628624939, Accuracy=0.6666666666666666
Epoch 951/1000, Loss=

IndexError: tuple index out of range

In [46]:
df = pd.read_csv('../data/train.csv')
df.fillna(df.median(), inplace=True)
X = df.drop(columns=['Name', 'Ticket', 'Cabin', 'SibSp', 'Parch', 'Embarked'])
y = X['Survived']
X = X.drop(columns=['Survived'])
X = pd.get_dummies(X, columns=['Sex'])

nn = NeuralNetwork(input_nodes=6,
                   hidden_nodes=[32, 32],
                   output_nodes=1,
                   batch_size=1,
                   learning_rate=1e-2,
                   momentum=0)

params, cost, accuracy = nn.train(X.values, y.values, epochs=1000)

# y_pred = nn.predict(X.values[0:10])
# print(y_pred)

Epoch 0/1000, Loss=0.11926388756195824, Accuracy=0.6094276094276094
Epoch 1/1000, Loss=0.11843062926473268, Accuracy=0.6161616161616161
Epoch 2/1000, Loss=0.11836417411430417, Accuracy=0.6161616161616161
Epoch 3/1000, Loss=0.11834346195256382, Accuracy=0.6161616161616161
Epoch 4/1000, Loss=0.11833228394231207, Accuracy=0.6161616161616161
Epoch 5/1000, Loss=0.11832442678551684, Accuracy=0.6161616161616161
Epoch 6/1000, Loss=0.11831925907854557, Accuracy=0.6161616161616161
Epoch 7/1000, Loss=0.11831604250873852, Accuracy=0.6161616161616161
Epoch 8/1000, Loss=0.11831416810081918, Accuracy=0.6161616161616161
Epoch 9/1000, Loss=0.11831308570454427, Accuracy=0.6161616161616161
Epoch 10/1000, Loss=0.11831327836555265, Accuracy=0.6161616161616161
Epoch 11/1000, Loss=0.11831318642845992, Accuracy=0.6161616161616161
Epoch 12/1000, Loss=0.11831295001868654, Accuracy=0.6161616161616161
Epoch 13/1000, Loss=0.11831293012964592, Accuracy=0.6161616161616161
Epoch 14/1000, Loss=0.11831336758633147, Acc

/home/rionaldichandraseta/miniconda3/envs/gemastik/lib/python3.6/site-packages/ipykernel_launcher.py:132: RuntimeWarning: overflow encountered in exp


Epoch 44/1000, Loss=0.11832169919611514, Accuracy=0.6161616161616161
Epoch 45/1000, Loss=0.11832026723382941, Accuracy=0.6161616161616161
Epoch 46/1000, Loss=0.118320037100953, Accuracy=0.6161616161616161
Epoch 47/1000, Loss=0.11832039511310037, Accuracy=0.6161616161616161
Epoch 48/1000, Loss=0.11832234106782472, Accuracy=0.6161616161616161
Epoch 49/1000, Loss=0.1183212757229232, Accuracy=0.6161616161616161
Epoch 50/1000, Loss=0.11832179066062568, Accuracy=0.6161616161616161
Epoch 51/1000, Loss=0.11832202549531473, Accuracy=0.6161616161616161
Epoch 52/1000, Loss=0.11832253402387963, Accuracy=0.6161616161616161
Epoch 53/1000, Loss=0.11832273652328272, Accuracy=0.6161616161616161
Epoch 54/1000, Loss=0.11832323093706176, Accuracy=0.6161616161616161
Epoch 55/1000, Loss=0.11832350318338475, Accuracy=0.6161616161616161
Epoch 56/1000, Loss=0.11832390143818784, Accuracy=0.6161616161616161
Epoch 57/1000, Loss=0.11832407055927409, Accuracy=0.6161616161616161
Epoch 58/1000, Loss=0.118324504846260

Epoch 163/1000, Loss=0.11832729415311681, Accuracy=0.6161616161616161
Epoch 164/1000, Loss=0.11832499582276315, Accuracy=0.6161616161616161
Epoch 165/1000, Loss=0.11832510060261525, Accuracy=0.6161616161616161
Epoch 166/1000, Loss=0.11832489757234475, Accuracy=0.6161616161616161
Epoch 167/1000, Loss=0.11832482630576917, Accuracy=0.6161616161616161
Epoch 168/1000, Loss=0.1183246193784198, Accuracy=0.6161616161616161
Epoch 169/1000, Loss=0.11832453658989055, Accuracy=0.6161616161616161
Epoch 170/1000, Loss=0.1183243035444322, Accuracy=0.6161616161616161
Epoch 171/1000, Loss=0.11832418965756143, Accuracy=0.6161616161616161
Epoch 172/1000, Loss=0.11832383168091344, Accuracy=0.6161616161616161
Epoch 173/1000, Loss=0.11832371410312803, Accuracy=0.6161616161616161
Epoch 174/1000, Loss=0.11832339069586867, Accuracy=0.6161616161616161
Epoch 175/1000, Loss=0.11832309255675755, Accuracy=0.6161616161616161
Epoch 176/1000, Loss=0.11832281928318383, Accuracy=0.6161616161616161
Epoch 177/1000, Loss=0

Epoch 281/1000, Loss=0.11823504012950736, Accuracy=0.6161616161616161
Epoch 282/1000, Loss=0.11823291315222657, Accuracy=0.6161616161616161
Epoch 283/1000, Loss=0.11823327118385739, Accuracy=0.6161616161616161
Epoch 284/1000, Loss=0.11823436587971148, Accuracy=0.6161616161616161
Epoch 285/1000, Loss=0.11823051952506014, Accuracy=0.6161616161616161
Epoch 286/1000, Loss=0.11823176632028981, Accuracy=0.6161616161616161
Epoch 287/1000, Loss=0.11822672959863842, Accuracy=0.6161616161616161
Epoch 288/1000, Loss=0.11822771993709368, Accuracy=0.6161616161616161
Epoch 289/1000, Loss=0.1182306414102179, Accuracy=0.6161616161616161
Epoch 290/1000, Loss=0.11822917532303409, Accuracy=0.6161616161616161
Epoch 291/1000, Loss=0.11822812227637695, Accuracy=0.6161616161616161
Epoch 292/1000, Loss=0.11822978813353888, Accuracy=0.6161616161616161
Epoch 293/1000, Loss=0.11822870216549236, Accuracy=0.6161616161616161
Epoch 294/1000, Loss=0.11822897004870615, Accuracy=0.6161616161616161
Epoch 295/1000, Loss=

...

Epoch 518/1000, Loss=0.11811096824676151, Accuracy=0.6184062850729517
Epoch 519/1000, Loss=0.11811006048174756, Accuracy=0.6184062850729517
Epoch 520/1000, Loss=0.11810987593609047, Accuracy=0.6184062850729517
Epoch 521/1000, Loss=0.118107377173684, Accuracy=0.6184062850729517
Epoch 522/1000, Loss=0.11810621186417633, Accuracy=0.6184062850729517
Epoch 523/1000, Loss=0.11810620070863961, Accuracy=0.6184062850729517
Epoch 524/1000, Loss=0.1181041283077529, Accuracy=0.6184062850729517
Epoch 525/1000, Loss=0.11810235517456445, Accuracy=0.6184062850729517
Epoch 526/1000, Loss=0.11810206097831666, Accuracy=0.6184062850729517
Epoch 527/1000, Loss=0.1180997003058682, Accuracy=0.6184062850729517
Epoch 528/1000, Loss=0.11809847557783067, Accuracy=0.6184062850729517
Epoch 529/1000, Loss=0.11809757803253644, Accuracy=0.6184062850729517
Epoch 530/1000, Loss=0.11809586657051091, Accuracy=0.6184062850729517
Epoch 531/1000, Loss=0.11809425645036015, Accuracy=0.6184062850729517
Epoch 532/1000, Loss=0.1

Epoch 636/1000, Loss=0.11784598650105509, Accuracy=0.6195286195286195
Epoch 637/1000, Loss=0.11783969322154478, Accuracy=0.6195286195286195
Epoch 638/1000, Loss=0.11784027813281409, Accuracy=0.6195286195286195
Epoch 639/1000, Loss=0.11783694224112509, Accuracy=0.6195286195286195
Epoch 640/1000, Loss=0.11783082657784942, Accuracy=0.6195286195286195
Epoch 641/1000, Loss=0.11783319256039919, Accuracy=0.6195286195286195
Epoch 642/1000, Loss=0.11783086751207214, Accuracy=0.6195286195286195
Epoch 643/1000, Loss=0.11782733454112941, Accuracy=0.6195286195286195
Epoch 644/1000, Loss=0.11781653988425173, Accuracy=0.6195286195286195
Epoch 645/1000, Loss=0.11782134221169005, Accuracy=0.6195286195286195
Epoch 646/1000, Loss=0.11782003805899287, Accuracy=0.6195286195286195
Epoch 647/1000, Loss=0.11781710178087022, Accuracy=0.6195286195286195
Epoch 648/1000, Loss=0.11781474498495527, Accuracy=0.6195286195286195
Epoch 649/1000, Loss=0.11781150703583171, Accuracy=0.6206509539842873
Epoch 650/1000, Loss

Epoch 755/1000, Loss=0.1176228985708466, Accuracy=0.621773288439955
Epoch 756/1000, Loss=0.11762741715347501, Accuracy=0.621773288439955
Epoch 757/1000, Loss=0.11762601817778032, Accuracy=0.621773288439955
Epoch 758/1000, Loss=0.1176288938815914, Accuracy=0.621773288439955
Epoch 759/1000, Loss=0.11762705809571537, Accuracy=0.621773288439955
Epoch 760/1000, Loss=0.11762483033842186, Accuracy=0.621773288439955
Epoch 761/1000, Loss=0.1176259353277912, Accuracy=0.621773288439955
Epoch 762/1000, Loss=0.11762218936940771, Accuracy=0.621773288439955
Epoch 763/1000, Loss=0.11762417034395799, Accuracy=0.621773288439955
Epoch 764/1000, Loss=0.11761962422913719, Accuracy=0.621773288439955
Epoch 765/1000, Loss=0.11762059243051468, Accuracy=0.621773288439955
Epoch 766/1000, Loss=0.11761793660426001, Accuracy=0.621773288439955
Epoch 767/1000, Loss=0.11761720507087778, Accuracy=0.621773288439955
Epoch 768/1000, Loss=0.11761517788532254, Accuracy=0.621773288439955
Epoch 769/1000, Loss=0.11761607634201

Epoch 874/1000, Loss=0.1174854329384668, Accuracy=0.622895622895623
Epoch 875/1000, Loss=0.11748437205693213, Accuracy=0.6240179573512907
Epoch 876/1000, Loss=0.1174832787182288, Accuracy=0.6240179573512907
Epoch 877/1000, Loss=0.11748245387827963, Accuracy=0.6240179573512907
Epoch 878/1000, Loss=0.11748063632591396, Accuracy=0.6240179573512907
Epoch 879/1000, Loss=0.11748019733216473, Accuracy=0.622895622895623
Epoch 880/1000, Loss=0.11747880860546416, Accuracy=0.6240179573512907
Epoch 881/1000, Loss=0.11747715389671906, Accuracy=0.622895622895623
Epoch 882/1000, Loss=0.11747637890018904, Accuracy=0.6240179573512907
Epoch 883/1000, Loss=0.11747480526775225, Accuracy=0.622895622895623
Epoch 884/1000, Loss=0.11747345797403602, Accuracy=0.6240179573512907
Epoch 885/1000, Loss=0.11747253315774756, Accuracy=0.622895622895623
Epoch 886/1000, Loss=0.11747111689324083, Accuracy=0.6240179573512907
Epoch 887/1000, Loss=0.11746902640341658, Accuracy=0.622895622895623
Epoch 888/1000, Loss=0.11746

Epoch 993/1000, Loss=0.11740171315479357, Accuracy=0.6240179573512907
Epoch 994/1000, Loss=0.11733982822515754, Accuracy=0.6240179573512907
Epoch 995/1000, Loss=0.11739345003929513, Accuracy=0.6240179573512907
Epoch 996/1000, Loss=0.11731915082726185, Accuracy=0.6240179573512907
Epoch 997/1000, Loss=0.11733260727256013, Accuracy=0.622895622895623
Epoch 998/1000, Loss=0.11733249901053618, Accuracy=0.622895622895623
Epoch 999/1000, Loss=0.11733908938906255, Accuracy=0.622895622895623


In [ ]:
nn.predict()